### PROJECT 1

Docelowy notebook w którym będzie finalny kod :-)

In [1]:
# imports
import torch.nn as nn
import torch.optim as optim
import numpy as np
from source.utils.config_manager import ConfigManager

# hyperparameters tuning
from ray import tune
from ray.tune import JupyterNotebookReporter
from ray.tune.schedulers import ASHAScheduler

from source.hyperparams_tuning import HyperparameteresTunner

# transformers
from source.transformers import TestTransformersFactory
from source.transformers import TrainTrasformersFactory

# data loaders
from source.dataloaders.cnn_data_loaders_factory import CNNDataLoaderFactory

# networks
from source.custom_cnn.pytorch_ray_tutorial_cnn import Net

In [2]:
dataset_name = 'cifar10'
dataset_path = ConfigManager.get_dataset_path(dataset_name)
gpus_per_trial = 2
max_num_epochs = 10
num_samples = 10
local_dir = ConfigManager.get_tuning_results_path()
    
config_criterion_1 = {
    "type": tune.choice([nn.CrossEntropyLoss])
}

config_optimizer_1 = {
    "type": tune.choice([optim.SGD]),
    "lr": tune.loguniform(1e-4, 1e-1),
}

config_net_1 = {
    "type": tune.choice([Net]),
    "optimizer": tune.choice([config_optimizer_1]),
    "criterion": tune.choice([config_criterion_1]),
    "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
    "l2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
}

config_net_2 = {
    "type": tune.choice([Net]),
    "optimizer": tune.choice([config_optimizer_1]),
    "criterion": tune.choice([config_criterion_1]),
    "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
    "l2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
}

config = {
    "data_loaders_factory": tune.choice([
        CNNDataLoaderFactory(dataset_path,
            TrainTrasformersFactory.get_train_transformer(), TestTransformersFactory.get_test_transformer())
    ]),
    "batch_size": tune.choice([2, 4, 8, 16]),
    "epochs": 10,
    "net": tune.choice([config_net_1, config_net_2]),
    "tuning_id": "XYZ"
}

scheduler = ASHAScheduler(
    metric="loss",
    mode="min",
    max_t=max_num_epochs,
    grace_period=1,
    reduction_factor=2)

reporter = JupyterNotebookReporter(max_progress_rows=20, overwrite=True, max_report_frequency=10, max_error_rows=5)

In [3]:
tuner = HyperparameteresTunner()

In [4]:
tuner.tune(dataset_name, local_dir, config, scheduler, reporter)

Trial name,status,loc,batch_size,data_loaders_factory,net,net/criterion,net/criterion/type,net/l1,net/l2,net/optimizer,net/optimizer/lr,net/optimizer/type,net/type,iter,total time (s),loss,accuracy
__train_validate_1218d_00000,TERMINATED,127.0.0.1:17160,16,<source.dataloaders.cnn_data_loaders_factory.CNNDataLoaderFactory object at 0x000001E1B9974BB0>,"{'type': <class 'source.custom_cnn.pytorch_ray_tutorial_cnn.Net'>, 'optimizer': {'type': <class 'torch.optim.sgd.SGD'>, 'lr': 0.07542107625690371}, 'criterion': {'type': <class 'torch.nn.modules.loss.CrossEntropyLoss'>}, 'l1': 128, 'l2': 16}",{'type': <class 'torch.nn.modules.loss.CrossEntropyLoss'>},<class 'torch.nn.modules.loss.CrossEntropyLoss'>,128,16,"{'type': <class 'torch.optim.sgd.SGD'>, 'lr': 0.07542107625690371}",0.0754211,<class 'torch.optim.sgd.SGD'>,<class 'source.custom_cnn.pytorch_ray_tutorial_cnn.Net'>,10,832.974,1.39089,0.520232


2022-03-15 14:11:06,688	INFO tune.py:639 -- Total run time: 989.30 seconds (840.86 seconds for the tuning loop).


Best trial config: {'data_loaders_factory': <source.dataloaders.cnn_data_loaders_factory.CNNDataLoaderFactory object at 0x000001E1B9974BB0>, 'batch_size': 16, 'epochs': 10, 'net': {'type': <class 'source.custom_cnn.pytorch_ray_tutorial_cnn.Net'>, 'optimizer': {'type': <class 'torch.optim.sgd.SGD'>, 'lr': 0.07542107625690371}, 'criterion': {'type': <class 'torch.nn.modules.loss.CrossEntropyLoss'>}, 'l1': 128, 'l2': 16}, 'tuning_id': 'XYZ_15-03-2022_13-54-24'}
Best trial final validation loss: 1.3908878765427148
Best trial final validation accuracy: 0.5202323717948718


__train_validate_1218d_00000